# Tarefa Prática - Problema do caixeiro viajante

Você é um viajante e deseja visitar todas as cidades de um país. Você tem um mapa com as distâncias entre todas as cidades. O problema é determinar a rota mais curta que começa em uma cidade, visita todas as outras cidades exatamente uma vez e retorna à cidade de origem. Implemente um algoritmo genético para resolver esse problema.

## Representação das distâncias

Considere um país com 5 cidades nomeadas de A a E. O mapa de distâncias entre as cidades é dado pela matriz abaixo:

|   | A | B | C | D | E |
|---|---|---|---|---|---|
| A | 0 | 12 | 3 | 23 | 1 |
| B | 12 | 0 | 9 | 18 | 3 |
| C | 3 | 9 | 0 | 89 | 56 |
| D | 23 | 18 | 89 | 0 | 45 |
| E | 1 | 3 | 56 | 45 | 0 |

## Instruções Adicionais

Para resolver o problema, será necessário:

 - Considerar a matriz de distâncias fornecida acima.
 - Inicializar a população de soluções de forma aleatória (cada indivíduo é uma permutação das cidades).
 - Implementar a função *fitness* para calcular a distância total percorrida por um indivíduo. Use a função inversa da distância total do percurso como função de *fitness*.
 - Implementar a função de seleção de pais por torneio para escolher os indivíduos que irão gerar a próxima geração.
 - Implementar a função de cruzamento para gerar novos indivíduos a partir dos pais selecionados. Utilize o operador de cruzamento de ordem ou outro operador de cruzamento adequado.
 - Implementar a função de mutação para alterar aleatoriamente um indivíduo da população. Use um valor de probabilidade de mutação adequado.
 - (opcional) Implementar a função de elitismo para manter os melhores indivíduos da população atual na próxima geração.

In [ ]:
import random, math

# Matriz com as distâncias entre cidades (A=0, B=1, C=2, D=3, E=4)
distancias = [
    [0, 12, 3, 23, 1],
    [12, 0, 9, 18, 3],
    [3, 9, 0, 89, 56],
    [23, 18, 89, 0, 45],
    [1, 3, 56, 45, 0]
]

# Lista das cidades representadas por números
cidades = list(range(len(distancias)))

#Fitness

In [ ]:
def calcular_distancia(caminho):
    distancia = 0
    for i in range(len(caminho)):
        origem = caminho[i]
        destino = caminho[(i + 1) % len(caminho)]  # O % garante que volta ao início
        distancia += distancias[origem][destino]
    return distancia

def fitness(caminho):
    return 1 / calcular_distancia(caminho)         # Inverso da distância

#População inicial

In [ ]:
def gerar_populacao(tamanho, cidades):  # Cada indivíduo é uma lista (permutação) de cidades visitadas em ordem.
    populacao = []
    for _ in range(tamanho):
        individuo = cidades[:]
        random.shuffle(individuo)
        populacao.append(individuo)     # embaralha a ordem (gera um caminho aleatório)
    return populacao

#Seleção por torneio

In [ ]:
def selecao_torneio(populacao, k=3):    # Seleciona o melhor indivíduo entre 'k' escolhidos aleatoriamente.
    competidores = random.sample(populacao, k)
    competidores.sort(key=lambda ind: fitness(ind), reverse=True)
    return competidores[0]              # retorna o melhor

#Crossover

In [ ]:
def crossover(pai, mae):
    tamanho = len(pai)
    inicio, fim = sorted(random.sample(range(tamanho), 2))

    filho = [None] * tamanho
    filho[inicio:fim] = pai[inicio:fim]

    pos = fim
    for gene in mae:
        if gene not in filho:    # só adiciona cidades ainda não incluídas
            if pos >= tamanho:   # se chegar no fim, volta para o início
                pos = 0
            filho[pos] = gene
            pos += 1
    return filho

#Mutação

In [ ]:
def mutacao(individuo, taxa=0.1):
    if random.random() < taxa:
        i, j = random.sample(range(len(individuo)), 2)
        individuo[i], individuo[j] = individuo[j], individuo[i]
    return individuo

#Elitismo

In [ ]:
def algoritmo_genetico(geracoes=500, tam_pop=20, taxa_mutacao=0.1, elitismo=True):
    populacao = gerar_populacao(tam_pop, cidades)

    for g in range(geracoes):
        nova_populacao = []

        if elitismo:   # Elitismo: mantém o melhor indivíduo da geração atual
            populacao.sort(key=lambda ind: fitness(ind), reverse=True)
            elite = populacao[0]
            nova_populacao.append(elite)

        while len(nova_populacao) < tam_pop:     # Gera novos indivíduos até completar a população
            pai = selecao_torneio(populacao)
            mae = selecao_torneio(populacao)
            filho = crossover(pai, mae)
            filho = mutacao(filho, taxa_mutacao)
            nova_populacao.append(filho)

        populacao = nova_populacao

    melhor = max(populacao, key=lambda ind: fitness(ind))
    return melhor, calcular_distancia(melhor)    # retorna o melhor indivíduo da última geração

# Testando

In [ ]:
melhor_caminho, melhor_distancia = algoritmo_genetico()
print("Melhor caminho (cidades como números):", melhor_caminho)
print("Distância:", melhor_distancia)

# Traduzido para letras
nomes = ['A', 'B', 'C', 'D', 'E']
print("Melhor caminho (cidades como letras):", [nomes[i] for i in melhor_caminho])

Melhor caminho (cidades como números): [4, 3, 1, 2, 0]
Distância: 76
Melhor caminho (cidades como letras): ['E', 'D', 'B', 'C', 'A']
